In [1]:
import json
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk

In [2]:
with open("reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [3]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

{"reviewerID": "AAXUNK0W2DZG5", "asin": "0060520841", "reviewerName": "Amazon Customer \"leneker\"", "helpful": [5, 10], "reviewText": "1996 Bernard Goldberg wrote an editorial for the Wall Street Journal that said there was an obvious bias on the part of network new shows  for the liberal point of view.  he then illustrated this with an example that he dissected in-depth.  The reaction to this observation was the ruination of her career, and the beginning of his status as a pariah among most newsmen.  This book is used to add more ammo to the controversy.That the journalists who so eagerly pry into other peoples lives and business should be reluctant to be examined is hardly surprising.  Almost no one really wants to have his life or business dissected by Mike Wallace not even Dan Rather.  Some facts in this book are really potent such as the survey results which show how the average journalist and the average American are often vastly at odds.  Other chapters highlight different stor

In [4]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)
type(review_data)

list

In [5]:
import random
random_data = random.sample(review_data,len(review_data))

In [6]:
reviewText = []
for i in range(len(random_data)):
    reviewText.append(random_data[i]["reviewText"])

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samsung\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# Stopword
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

stopword = []
for i in range(len(reviewText)):
    Lower_reviewText = [j for j in reviewText[i].lower().split() if j not in stop]
    Join_Lower_reviewText = " ".join(Lower_reviewText)
    stopword.append(Join_Lower_reviewText)

In [ ]:
print(stopword[428])

thirteen-year-old millicent maccool knows fourteenth birthday ordinary day. like fourteen-year-old's birthdays. no, all. see, millicent's family, turning fourteen means receiving magical gift, magical gift, ability make one wish life come true. sure millicent wants grow-up, she's sure she's ready kind power. day fourteen bithday draws closer closer, millicent slowly realizing dangerous one gift be, difficult choices face throughout life be, well. millicent learning struggly love, secrets, betrayal, end, millicent make hardest decision ever make. one change life forever, knows it.i surprised stumble across ann rinaldi's millicent's gift, browsing bookstore day, accustomed writing historical fiction, millicent's gift nothing like that. instead, paranormal/magic book set modern day times. however, applaud ms. rinaldi amazing work, millicent's gift wonderful work literature young old. millicent exciting character, wonderful friend, trustworthy daughter, niece, sister. intelligent, determin

In [23]:
# # Stemming

# stemming = []
# from nltk.stem.porter import PorterStemmer
# porter = PorterStemmer()

# def tokenizer_porter(text):
#     return [porter.stem(word) for word in text.split()]

# for i in range(len(stopword)):
#     try:
#         Stemming = tokenizer_porter(stopword[i])
#         Join_Stemming = " ".join(Stemming)
#         stemming.append(Join_Stemming)
#     except:
#         print(i)

In [ ]:
# Tagging

def pos_tagging(document):
    result = nltk.pos_tag( nltk.word_tokenize(document))    
    last_result = ['/'.join(t) for t in result]
    return last_result

tag = [pos_tagging(i) for i in stopword]

In [ ]:
len(tag)

In [ ]:
# JJ분류
tagging_JJ = []
for i in range(len(tag)):
    tagging = []
    for j in tag[i]:
        if j.split("/")[1] == "RB":
            tagging.append(j.split("/")[0])
    join_each_tag = " ".join(tagging)
    tagging_JJ.append(join_each_tag)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, max_df = 0.3, min_df = 2, lowercase=True, stop_words = 'english')
D = vectorizer.fit_transform(tagging_JJ)

In [ ]:
D.shape

In [ ]:
y_data = []
int_y_data = []
for i in range(len(random_data)):
    y_data.append(random_data[i]["overall"])
y_data=np.array(y_data)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(D, y_data, test_size=0.2, random_state=22)

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

MultinomialNB

In [34]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.05, class_prior=None, fit_prior=True)
clf.fit(x_train, y_train)

MultinomialNB(alpha=0.05, class_prior=None, fit_prior=True)

In [35]:
result = clf.predict(x_test)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(result,y_test)

0.56774158931158447

SVM

In [59]:
from sklearn.svm import SVC
clf= SVC(C=1500, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
clf.fit(x_train, y_train) 

SVC(C=1500, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [60]:
result = clf.predict(x_test)

In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(result,y_test)

0.56489778397810808

Multiclass SVM (OneVsRestClassifier)

In [ ]:
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

result = OneVsRestClassifier(LinearSVC(random_state=0)).fit(x_train, y_train).predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(result,y_test)

LogisticRegression

In [ ]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression()

logreg.fit(x_train, y_train)
logreg.predict(x_test) == y_test

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(x_test), y_test)

GaussianNB

In [50]:
# from sklearn.naive_bayes import GaussianNB
# clf = GaussianNB()
# clf.fit(x_train, y_train)

In [51]:
# result = clf.predict(x_test)

In [52]:
# from sklearn.metrics import accuracy_score
# accuracy_score(result,y_test)